<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

from spektral.datasets import citation
from spektral.layers import GraphConv

In [2]:
# load data
dataset = 'cora'
A, X, y, train_mask, val_mask, test_mask = citation.load_data('cora')

# Parameters
channels = 16           # Number of channels in the first layer
N = X.shape[0]          # Number of nodes in the graph
F = X.shape[1]          # Original size of node features
n_classes = y.shape[1]  # Number of classes
dropout = 0.5           # Dropout rate for the features
l2_reg = 5e-4 / 2       # L2 regularization rate
learning_rate = 1e-2    # Learning rate
epochs = 200            # Number of training epochs
es_patience = 10        # Patience for early stopping


Loading cora dataset
Pre-processing node features


In [ ]:
# preprocess
fltr = GraphConv.preprocess(A).astype('f4')
X = X.toarray()

In [3]:
A, X, y, train_mask, val_mask, test_mask = citation.load_data('cora')


Loading cora dataset
Pre-processing node features


In [4]:
print(type(A),type(X),type(y),type(train_mask),)

<class 'scipy.sparse.csr.csr_matrix'> <class 'scipy.sparse.csr.csr_matrix'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [5]:
A.

<2708x2708 sparse matrix of type '<class 'numpy.int64'>'
	with 10556 stored elements in Compressed Sparse Row format>

should i just use pytorch instead?
now that i know how label propagation works a little better, should i still start with chemical prediction (given i don't have a background in the topic?) or move to a graph I know a little more about

Loading cora dataset
Pre-processing node features


In [4]:
X.shape

(2708, 1433)

In [5]:
X.shape[-1]

1433

In [9]:
type(X[0])

scipy.sparse.csr.csr_matrix

Node citation network. each node is a document, edges are citations (undirected).
each node has a binary BoW attribute (1 if word is in text). each node has class label for prediction
- transductive learning
    - observe all nodes & edges at training time, but only some labels
    - goal: predict missing labels
        - 'infer topic of unknown paper based on citation links'


In [22]:
# Keras API & Graph Convolutional layer
from spektral.layers import GraphConv
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout

from spektral.datasets import citation
A, X, y, train_mask, val_mask, test_mask = citation.load_data('cora')
# Adj_matrix = A, sparse matrix of (n,n) node_num
# node features X shape(N,F)
# labels y, shape (N, n_classes)
N = A.shape[0]
F = X.shape[-1]
n_classes = y.shape[-1]

Loading cora dataset
Pre-processing node features


In [28]:
train_mask

array([ True,  True,  True, ..., False, False, False])

In [23]:
# provide multiple inputs, X and A, to GraphConv layers
# requires graph & its adj_matrix

# dataset is just nodes & edges; keras must consider each node as separate sample
# so batch axis is None
X_in = Input(shape=(F, ))
A_in = Input((N, ), sparse=True)
# sampling node attributes = vector, shape(F, )
# sampling adj_matrix = vector, shape(N, )

X_1 = GraphConv(16, 'relu')([X_in, A_in])
# regularize with dropout layer
X_1 = Dropout(0.5)(X_1)
X_2 = GraphConv(n_classes, 'softmax')([X_1, A_in])

model = Model(inputs=[X_in, A_in], outputs=X_2)

In [24]:
# pre-proc adj_matrix to add self loops + scale weights of node's connections based on degree
A = GraphConv.preprocess(A).astype('f4') # depends on type of layer
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             weighted_metrics=['acc'])
# weighted_metrics is better than metrics for our semi-supervised problem (boolean masks)
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 1433)]       0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 2708)]       0                                            
__________________________________________________________________________________________________
graph_conv_4 (GraphConv)        (None, 16)           22944       input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 16)           0           graph_conv_4[0][0]    

In [25]:
# prep data
X = X.toarray()
A = A.astype('f4')
validation_data = ([X, A], y, val_mask)
# train
# batch_size=N, shuffle=False. default Keras splits data into batches of 32, shuffles at epoch
# shuffling our adj_matrix on one axis and not other will disjoint our rows completely
# splitting graph into batches (subgraphs) might cause havoc and "disconnected" nodes
# take all node features at once to avoid this.
model.fit([X, A], y,
          sample_weight=train_mask,
          validation_data=validation_data,
          batch_size=N,
          shuffle=False,
         epochs = 100) # default 10 epochs
# train_mask, val_mask for sample_weight
# in training, train nodes will have weight of 1, val nodes have weight of 0
# in validation, train nodes have weight 0, val nodes have weight 1
 # so we only have to change the mask to differentiate train/test data
    # since the model takes X, A and y for train and val phases

In [27]:
# evaluate, only changing the sample_weight and batch_size like before
eval_results = model.evaluate([X, A],
                              y,
                              sample_weight=test_mask,
                              batch_size=N)
print('Done.\n'
      'Test loss: {}\n'
      'Test accuracy: {}'.format(*eval_results))

1/1 [==============================] - 0s 852us/step - loss: 0.6856 - acc: 0.7000
Done.
Test loss: 0.6856234073638916
Test accuracy: 0.699999988079071


In [29]:
n_classes

7